In [ ]:
import requests
import pandas as pd
import time

LIMIT = 50            # Giới hạn request tối đa
MAX_PAGES = 50        # Crawl tối đa 50 trang
CITY_CODE = 12000     # Mã code Thành phố Hà Nội
CATEGORY_CODE = 1000  # Mã danh mục để chọn đúng tin cần crawl
API_URL = "https://gateway.chotot.com/v1/public/ad-listing"

# Hàm lấy chi tiết tình trạng nội thất
def get_furniture(list_id):   # Lấy theo id từng tin một
    url = f"https://gateway.chotot.com/v1/public/ad-listing/{list_id}"
    try:
        res = requests.get(url, timeout=10)   # Gửi request
        if res.status_code != 200:            # Kiểm tra trạng thái HTTP
            return "Không rõ"
        data = res.json()                     # parse JSON chi tiết theo id

        # Kiểm tra trong params (nếu có)
        for p in data.get("params", []):
            key = str(p.get("id", "")).lower()
            if "furn" in key or "noi_that" in key or "interior" in key:
                return p.get("value_label") or p.get("value") or "Không rõ"

        # Một số tin có thể chứa trong ad_params
        for k, v in (data.get("ad_params") or {}).items():
            if "furn" in k.lower() or "noi_that" in k.lower():
                if isinstance(v, dict):
                    return v.get("value_label") or v.get("value")
                return v

        return "Không rõ"
    except Exception as e:    # Bắt lỗi mạng/JSON bất thường -> Trả về"Không rõ"
        print("Lỗi get_furniture:", e)
        return "Không rõ"

def crawl_nhatot():
    all_ads = []        # Lưu tạm kết quả
    seen_ids = set()    # Dùng set để tránh crawl bị lặp

    for i in range(1, MAX_PAGES + 1):   # Lặp theo offset
        o = LIMIT * (i - 1)
        url = f"{API_URL}?region_v2={CITY_CODE}&cg={CATEGORY_CODE}&o={o}&st=s,k&limit={LIMIT}&w=1&include_expired_ads=true&key_param_included=true&video_count_included=true"
        res = requests.get(url, timeout=10)   # Gửi request

        if res.status_code != 200:            # Kiểm tra trạng thái HTTP
            print(f"Lỗi {res.status_code} ở trang {i}")
            continue

        ads = res.json().get("ads", [])
        if not ads:   # Nếu mảng rỗng tức là hết dữ liệu -> Dừng
            print(f"Trang {i} không có dữ liệu, dừng lại.")
            break

        new_ads = [ad for ad in ads if ad["ad_id"] not in seen_ids]   # Lọc những tin chưa thấy
        for ad in new_ads:
            seen_ids.add(ad["ad_id"])       # Đánh dấu là đã xử lý

            # Ghép địa chỉ chi tiết (phường + quận + thành phố)
            ward = ad.get("ward_name", "")
            area = ad.get("area_name", "")
            region = ad.get("region_name", "")
            full_address = ", ".join([x for x in [ward, area, region] if x])

            all_ads.append({
                "Ngày đăng": ad.get("list_time"),                           # Lấy time theo milliseconds
                "Loại hình căn hộ": ad.get("category_name"),
                "Diện tích (m2)": ad.get("size"),
                "Giá (VND)": ad.get("price"),
                "Giấy tờ pháp lý": ad.get("property_legal_document"),
                "Số phòng ngủ": ad.get("rooms"),
                "Số phòng vệ sinh": ad.get("toilets"),
                "Tình trạng nội thất": get_furniture(ad.get("list_id")),    # Lấy tình trạng nội thất theo lisi_id
                "Tiêu đề": ad.get("subject"),
                "Địa chỉ": full_address
            })

        print(f"Trang {i}: lấy {len(new_ads)} mẫu mới (Tổng: {len(all_ads)})")
        time.sleep(0.5)

        if len(new_ads) == 0:
            print("Không còn tin mới, dừng crawl.")
            break

    # Tạo DataFrame với đúng thứ tự cột
    df = pd.DataFrame(all_ads, columns=[
        "Ngày đăng", "Loại hình căn hộ", "Diện tích (m2)", "Giá (VND)",
        "Giấy tờ pháp lý", "Số phòng ngủ", "Số phòng vệ sinh",
        "Tình trạng nội thất", "Tiêu đề", "Địa chỉ"
    ])

    # Xuất CSV
    df.to_csv("raw_data.csv", index=False, encoding="utf-8-sig")
    print(f"Crawl hoàn tất, tổng: {len(df)} tin. Đã lưu raw_data.csv")
    return df


# Gọi hàm crawl
df = crawl_nhatot()

# Chuyển timestamp sang dạng ngày giờ đầy đủ
df["Ngày đăng"] = pd.to_datetime(df["Ngày đăng"] / 1000, unit="s", errors="coerce")
df["Ngày đăng"] = df["Ngày đăng"].dt.strftime("%d/%m/%Y %H:%M:%S")

# Format giá
df["Giá (VND)"] = df["Giá (VND)"].apply(
    lambda x: f"{int(x):,}" if pd.notnull(x) and x != 0 else ""
)

# Lưu lại file sạch
df.to_csv("raw_data_P1.csv", index=False, encoding="utf-8-sig")

# Tải file về
from google.colab import files
files.download("raw_data_P1.csv")